In [60]:
import os
from pathlib import Path
from loguru import logger
import sys
from reload_recursive import reload_recursive
import mri_data
import monai_training

In [49]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import DataSet, scan_3Tpioneer_bids, parse_image_name, nifti_name
from mri_data import file_manager as fm
from monai_training.preprocess import DataSetProcesser

In [36]:
logger.remove()
logger.add(sys.stderr, level='INFO')

2

In [51]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
inference_root = drive_root / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"

workspace_root = "/home/srs-9/Projects/ms_mri/itksnap_workspaces"
dataset_name = "choroid_pineal_pituitary_predictions"

save_dir = os.path.join(workspace_root, dataset_name)

In [56]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [47]:
proc_inference_labels = DataSetProcesser.new_dataset(
    inference_root,
    scan_3Tpioneer_bids,
    filters=[fm.filter_first_ses]
)
proc_inference_labels.prepare_labels("choroid_resegment1.pineal1.pituitary1")
label_data = proc_inference_labels.dataset
label_data.sort()

2024-10-21 02:56:49.157 | INFO     | monai_training.preprocess:prepare_labels:146 - Prepare Labels
2024-10-21 02:56:50.984 | INFO     | monai_training.preprocess:prepare_labels:206 - Dataset size: 159


In [48]:
dataset = DataSet.dataset_like(label_data)
dataset.dataroot = dataroot
dataset.add_images("flair.t1.nii.gz")

In [50]:
def convert_to_winroot(path: Path):
    return Path("H:/") / path.relative_to("/mnt/h")

This is unnecessarily complicated. Instead of adding `flair.t1.nii.gz` as the image attribute then having to split it up, just hard code `t1 = scan.root / "t1.nii.gz".

Although the current approach may be more reusable, so think about it

In [63]:
def create_workspace(label_scan, image_scan):
    label_path = convert_to_winroot(label_scan.label_path)
    image_names = parse_image_name(image_scan.image)
    label_root = Path(label_scan.root)
    image_root = Path(image_scan.root)

    image_paths = [(image_root / name).with_suffix(".nii.gz") for name in image_names]
    image_paths = [convert_to_winroot(p) for p in image_paths]

    main_image = "-layers-set-main {} -tags-add {}-MRI".format(
        image_paths[0], image_names[0].upper()
    )
    extra_images = " ".join(
        [
            "-layers-add-anat {} -tags-add {}-MRI".format(path, name.upper())
            for path, name in zip(image_paths[1:], image_names[1:])
        ]
    )
    seg = "-layers-add-seg {} -tags-add {}".format(label_path, nifti_name(label_scan.label))

    save_path = os.path.join(save_dir, f"sub-ms{label_scan.subid}-ses-{label_scan.sesid}.itksnap")
    save = f"-o {save_path}"

    command_parts = ["itksnap-wt.exe", main_image, extra_images, seg, save]
    command = " ".join(command_parts)
    # run(command)
    return command

In [73]:
commands = []
for label_scan, image_scan in zip(label_data, dataset):
    command = create_workspace(label_scan, image_scan)
    commands.append(command + "\n")

In [74]:
current_dir = Path(os.getcwd())
with open(current_dir / "create-workspaces2.sh", "w") as f:
    f.writelines(commands)
